In [301]:
import numpy as np
import pandas as pd
import plotly.express as px

In [293]:
np.random.seed(0)

In [294]:
def best_response(x_0, thresholds, priors):
    X = np.arange(x_0, threshold_max, threshold_delta).round(3)
    utilities = np.empty_like(X)
    for i, x in enumerate(X):
        utility = 0.
        for threshold, prior in zip(thresholds, priors):
            utility += prior * (x >= threshold)
        utility -= abs(x - x_0)
        utilities[i] = utility

    return X[np.argmax(utilities)]

In [295]:
def bayesian_update(priors):
    if np.sum(priors) == 0:
        return np.zeros_like(priors)
    return priors / np.sum(priors)

In [296]:
def balance_priors(priors, random=False):
    total = np.sum(priors)
    if total == 1:
        return priors
    indices = priors == 0
    remainder = 1 - total
    if random:
        p = np.random.rand(indices.sum())
        p = (p / p.sum()) * remainder
    else:
        p = remainder / indices.sum()
    priors[indices] = p
    return priors

In [297]:
num_agents = 10_000
threshold_min, threshold_max, threshold_delta = 0., 1., 1e-3

In [ ]:
X = np.random.uniform(size=num_agents)
thresholds = np.arange(threshold_min, threshold_max, 0.1).round(1)

priors = np.zeros_like(thresholds)
priors[4] = 0.05
priors[5] = 0.3
priors[6] = 0.3
priors[7] = 0.1
# Set `random=True` to randomly distribute the remaining probabilities
# Set `random=False` to evenly distribute the remaining probabilities
balance_priors(priors, random=True)
assert np.sum(priors) == 1

pd.DataFrame({"threshold": thresholds, "priors": priors}).round(3)

,threshold,priors
0,0.0,0.036
1,0.1,0.033
2,0.2,0.049
3,0.3,0.068
4,0.4,0.050
5,0.5,0.300
6,0.6,0.300
7,0.7,0.100
8,0.8,0.028
9,0.9,0.036


In [299]:
threshold_true = 0.3
Y_true = X > threshold_true

partitions = [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]

accuracy_loss = np.empty_like(thresholds)
for i, partition in enumerate(partitions):
    # Partition thresholds and their respective priors
    threshold_partition = thresholds[partition]
    priors_partition = priors[partition]
    # Update priors
    priors_partition = bayesian_update(priors_partition)
    
    # Compute best response
    X_r = np.empty_like(X)
    for xi, x_0 in enumerate(X):
        X_r[xi] = best_response(x_0, threshold_partition, priors_partition)
    # Compute predicted labels
    Y_pred = X_r > threshold_true
    # Compute accuracy loss
    accuracy_loss[i] = np.sum(Y_pred != Y_true) / Y_true.shape[0]

In [300]:
px.line(x=thresholds, y=accuracy_loss, labels={"x": "threshold", "y": "accuracy_loss"}, markers=True)